In [1]:
from bs4 import BeautifulSoup
import requests
import yaml

In [2]:
R = requests.get("https://omeka.org/classic/plugins/")

In [3]:
soup = BeautifulSoup(R.text, "html.parser")

In [4]:
entries = soup.select(".addon-entry")

In [5]:
def get_description_url(element):
    link = element.find('h4').find('a').get('href')
    if link[:4] != 'http':
        link = f'https://omeka.org/classic/plugins/{link}'
    return link

In [6]:
entryInfo = {}
for e in entries:
    eInfo = {
        'name': e.find('h4').get_text(),
        'description': e.find(attrs={'class':'description'}).get_text(),
        'download_url': e.find('a', attrs={'class':'button'}).get('href'),
        'description_url': get_description_url(e),
        'version': e.find(attrs={'class':'version'}).get_text(),
        'updated': e.find(attrs={'class':'date'}).get_text(),
    }
    entryInfo[eInfo['name']] = eInfo

In [7]:
for key,e in entryInfo.items():
    e['version'] = e['version'].replace('Latest Version: ','')
    e['updated'] = e['updated'].replace('Updated: ', '')

In [8]:
with open('all_plugin_info.yml', 'r') as file:
    originalEntryInfo = yaml.load(file, Loader=yaml.FullLoader)

In [9]:
for key, info in entryInfo.items():
    if key not in originalEntryInfo:
        originalEntryInfo[key] = info
    else:
        originalEntryInfo[key].update(info)

In [10]:
with open('all_plugin_info.yml', 'w') as file:
    documents = yaml.dump(originalEntryInfo, file)